In [1]:
import pandas as pd
import sqlite3
import json

################################################################################################################
# Initialize a dictionary to store ASN relationships
asn_relations = {}

# Open the file and read lines
with open('../manrs_2023-08/20230801.as-rel.txt', 'r') as file:
    relationships = file.readlines()

# Process lines
found_format_line = False

for entry in relationships:
    if entry.startswith("# step 1: set peering in clique"):
        continue
    if entry.startswith("# step 2: initial provider assignment"):
        continue
    if entry.startswith('# source:topology'):
        continue
    entry = entry.strip().split('|')
    if len(entry) == 1:
        # Handling for a single column
        continue
    asn = entry[1]
    data = entry[0]
    
    if asn not in asn_relations:
        # If encountering this ASN for the first time, initialize its relationships
        asn_relations[asn] = {
            'asn': asn,
            'providers': [],
            'customers': []
        }
    
        if entry[2] == '0':
            # If data is 0, add data to the customers list
            asn_relations[asn]['customers'].append(data)
        elif entry[2] == '-1':
            asn_relations[asn]['providers'].append(data)
    else:
        if entry[2] == '0':
            # If data is 0, add data to the customers list
            asn_relations[asn]['customers'].append(data)
        elif entry[2] == '-1':
            asn_relations[asn]['providers'].append(data)
              
# Convert the dictionary to a DataFrame
df_relations = pd.DataFrame(list(asn_relations.values()))
# Add the "id" column
df_relations.insert(0, 'id', range(1, len(df_relations) + 1))
df_relations['providers'] = df_relations['providers'].astype(str)
df_relations['customers'] = df_relations['customers'].astype(str)
##########################################################################################################

###########################################################################################################
# Read nro-delegated-stats data
nro_datas = []

with open('../manrs_2023-08/nro-delegated-stats', 'r') as file:
    nro_data_brut = file.readlines()
 
# Process lines
found_format_line = False

for line in nro_data_brut:
    if line.startswith("iana|ZZ|asn|0|1|20140311|reserved|ietf|iana"):
            found_format_line = True
            continue
    if found_format_line:
        nro_datas.append(line.strip().split('|'))
#registry|cc|type|start|value|date|status|opaque-id[|extensions...]       
header = ['registry', 'cc', 'type', 'start', 'value', 'date', 'status', 'opaque_id', 'extensions']

nro_datas_df = pd.DataFrame(nro_datas, columns=header)
# Add the "id" column
nro_datas_df.insert(0, 'id', range(1, len(nro_datas_df) + 1))
###########################################################################################################   
# Read ii.as-org.v01.2023-01.json data
with open('../manrs_2023-08/ii.as-org.v01.2023-01.json', 'r') as file:
    asorg_data = json.load(file)
    
# Create a dictionary to store the data
data_dict = {}

# Iterate through the keys and values of the JSON dictionary
for id, (asn, info) in enumerate(asorg_data.items(), start=1):
    data_dict[asn] = {
        "id": id,
        "asn": asn,
        "status": info["Status"],
        "reference_orgs": info["Reference Orgs"],
        "sibling_asns": info["Sibling ASNs"],
        "name": info["Name"],
        "descr": info["Descr"],
        "website": info["Website"],
        "comparison_with_ca2O": info["Comparison with CA2O"],
        "comparison_with_pdb": info["Comparison with PDB"],
        "pdb_org_id": info["PDB.org_id"],
        "pdb_org": info["PDB.org"]
    }

# Create a DataFrame from the dictionary
df_ii_as_org = pd.DataFrame.from_dict(data_dict, orient="index")   
df_ii_as_org['sibling_asns'] = df_ii_as_org['sibling_asns'].astype(str)
df_ii_as_org['reference_orgs'] = df_ii_as_org['reference_orgs'].astype(str)
############################################################################################################

##################################################################################################################
category_df = pd.read_csv('../manrs_2023-08/2023-05_categorized_ases.csv', dtype=str)
selected_columns = ['ASN', 'Category 1 - Layer 1', 'Category 1 - Layer 2']
category_df = category_df.loc[:, selected_columns]

header_mapping = {'ASN': 'asn',
                  'Category 1 - Layer 1': 'category_1',
                  'Category 1 - Layer 2': 'category_2'}
category_df = category_df.rename(columns=header_mapping)
category_df.insert(0, 'id', range(1, len(category_df) + 1))
category_df['asn'] = category_df['asn'].str.replace('AS', '')
##########################################################################################################################

# Create a dictionary of DataFrames with their associated table names
data_frames = {
    "delegated_stats": nro_datas_df,
    "dataset_as_mapping": df_ii_as_org,
    "categorized_asn": category_df,
    "relationship_asn": df_relations,
}

# Create a connection to the SQLite database
database_filename = "newdatabase.db"
conn = sqlite3.connect(database_filename)

# Iterate through the DataFrames and save each DataFrame as a table
for table_name, df in data_frames.items():
    df.to_sql(table_name, conn, if_exists="replace", index=False)

# Close the connection
conn.close()
